In [1]:
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real

2024-02-26 19:04:06.866431: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 19:04:06.917106: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 19:04:06.917162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 19:04:06.919024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 19:04:06.927908: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 19:04:06.931157: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

In [3]:
repeats = 1
seed = 1
save_data = False
at_class = atom_type.AT_Scheme_7()

In [4]:
molec_data_dict = {"R32":r14_class} #, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
# all_gp_dict = {"R14" : {list(all_gp_dict["R14"].keys())[0]: list(all_gp_dict["R14"].values())[0]}}
all_gp_dict = {"R32" : all_gp_dict["R32"]}
driver = opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)
ls_results = driver.optimize_ats()
ls_results

,Run,Iter,Min Obj,sigma_C1_min,sigma_C2_min,sigma_H1_min,sigma_F_H2_min,sigma_F_H1_min,sigma_F_Hx_min,epsilon_C1_min,...,R14-sim_vap_density-sse,R14-sim_Pvap-sse,R14-sim_Hvap-sse,R14-sim_liq_density-wt,R14-sim_vap_density-wt,R14-sim_Pvap-wt,R14-sim_Hvap-wt,Run Time,jac evals,Termination
0,1,1,1094.131300,2.834044,3.440649,1.500172,2.604665,2.293512,2.184677,22.106914,...,642796.500413,2053.910565,20337.330237,0.000037,0.000023,0.162610,0.037329,146.468565,80,0
1,1,2,1094.131021,2.834044,3.440649,1.500172,2.604665,2.293512,2.184677,22.106914,...,642796.439755,2053.910152,20337.324138,0.000037,0.000023,0.162610,0.037329,146.468565,80,0
2,1,3,1094.131300,2.834044,3.440649,1.500172,2.604665,2.293512,2.184677,22.106914,...,642796.500413,2053.910565,20337.330237,0.000037,0.000023,0.162610,0.037329,146.468565,80,0
3,1,4,1094.131300,2.834044,3.440649,1.500172,2.604665,2.293512,2.184677,22.106914,...,642796.500413,2053.910565,20337.330237,0.000037,0.000023,0.162610,0.037329,146.468565,80,0
4,1,5,1094.131300,2.834044,3.440649,1.500172,2.604665,2.293512,2.184677,22.106914,...,642796.500413,2053.910565,20337.330237,0.000037,0.000023,0.162610,0.037329,146.468565,80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,1,1036,0.142473,3.462138,3.440649,1.500172,2.604665,2.293512,2.911055,27.525330,...,14.938216,0.393882,1.406294,0.000858,0.000469,0.138229,0.060443,146.468565,80,0
1036,1,1037,0.142473,3.462138,3.440649,1.500172,2.604665,2.293512,2.911055,27.525330,...,14.938216,0.393882,1.406294,0.000858,0.000469,0.138229,0.060443,146.468565,80,0
1037,1,1038,0.142473,3.462138,3.440649,1.500172,2.604665,2.293512,2.911055,27.525330,...,14.938216,0.393882,1.406294,0.000858,0.000469,0.138229,0.060443,146.468565,80,0
1038,1,1039,0.142473,3.462138,3.440649,1.500172,2.604665,2.293512,2.911055,27.525330,...,14.938216,0.393882,1.406294,0.000858,0.000469,0.138229,0.060443,146.468565,80,0


In [5]:
ls_results.to_csv("test1.csv")

In [9]:
print(ls_results.iloc[-1])

Run                                 1
Iter                             1040
Min Obj                      0.142473
sigma_C1_min                 3.462138
sigma_C2_min                 3.440649
sigma_H1_min                 1.500172
sigma_F_H2_min               2.604665
sigma_F_H1_min               2.293512
sigma_F_Hx_min               2.911055
epsilon_C1_min               27.52533
epsilon_C2_min              32.461447
epsilon_H1_min                5.17414
epsilon_F_H2_min            33.858586
epsilon_F_H1_min            29.671808
epsilon_F_Hx_min            33.066857
Min Obj Cum.                 0.142473
sigma_C1_cum                 3.462119
sigma_C2_cum                 3.440649
sigma_H1_cum                 1.500172
sigma_F_H2_cum               2.604665
sigma_F_H1_cum               2.293512
sigma_F_Hx_cum               2.911057
epsilon_C1_cum              27.527017
epsilon_C2_cum              32.461447
epsilon_H1_cum                5.17414
epsilon_F_H2_cum            33.858586
epsilon_F_H1

In [7]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
driver = opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, save_data)